# Parser FINAL

Importamo potrebne knjižnice.

In [ ]:
import nltk
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import time

Spodaj je funkcija, ki prejme url in vrne sprasan naslov, telo in komentarje novice.

In [ ]:
def parse_url(url):
    try:
        driver = webdriver.Chrome('..\\webdriver\\chromedriver.exe')
        driver.get(url)
        button = driver.find_element_by_class_name('comment-show-hide')
        #webdriver.ActionChains(driver).move_to_element(button).click(button).perform()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        button.click()
        time.sleep(1)
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        #html = driver.page_source
        #print(html)
        driver.close()
        
        soup = BeautifulSoup(html, 'html.parser')
        newsContents = soup.find_all("div", id="newsbody")[0].find_all("p")
        parsed = ""
        
        #print(html)
        
        comments = soup.find_all("div", class_="newscomments")[0].find_all("dt", class_="ds2")
        #print("QWE: " + comments.get_text())
        for p in newsContents: 
            parsed += p.get_text() + " "
        title = soup.find_all("h1")[0].get_text()
        parsed_comments = [c.get_text() for c in comments]
        return parsed, title, parsed_comments
    except:
        print("Error url: " + url)
        return "", "", ""

Test klica funkcije.

In [ ]:
parse_url("http://www.rtvslo.si/svet/bliznji-vzhod/smrt-ameriki-iranci-protestirajo-proti-novim-sankcijam/470795")

Naredimo še funkcijo, ki s pomočjo Rtvslo iskalnika vrne vse povezave do najdenih novic.

In [ ]:
def get_urls(query, num_of_pages=1):
    urls = []
    for i in range(num_of_pages):
        html = urlopen("http://www.rtvslo.si/iskalnik?q=" + query + "&page=" + str(i)).read() 
        soup = BeautifulSoup(html, 'html.parser')
        for news in soup.find_all("div", class_="stitle"):
            urls.append("http://www.rtvslo.si" + news.a["href"])
    return urls

In [ ]:
get_urls("trump", 2)

Definiramo seznam novic, ki bi jih radi sparsali.

Za vsako novico kličemo prvo metodo.

In [ ]:
urls = get_urls("trump", 2)
parsed_news = []
for url in urls:
    parsed, title, comments = parse_url(url)
    if parsed != "":
        parsed_news.append((title, parsed, comments))
parsed_news

Zapišemo v datoteko.

In [ ]:
def write_to_file():
    f = open("../data/novice.txt", "w", encoding="utf-8")
    for title, parsed, comments in parsed_news:
        f.write(title + "\n")
        f.write(parsed + "\n")
        for c in comments:
            f.write(c.replace("\n", " ") + "\n")
        f.write("\n")

In [ ]:
write_to_file()